In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import numpy as np
import numpy
import pandas as pd
import json

cred = credentials.Certificate('key.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [2]:
data = {
        u'name': u'Tims Apple Place',
        u'state': u'First Road, 2004 Perth',
        u'country': u'timsfarm.co.au'
    }
addFarm(data)

In [34]:
def addFarm(data):
    # before adding, check if another value with the same Name & Adress exists. Right now we just add another
    dbRef = db.collection(u'farms').document()
    dbRef.set(data)

def addFarmsFromDF(df):
    #Example data
    """data = {
        u'id': df.Index[i],
        u'name': df.Name[i],
        u'adress': df.Adress[i],
        u'phone': df.Phone[i],
        u'website': df.Website[i],
        u'email': df.Email[i]
        }"""
    
    dataItems = {"id": "Index", "name": "Name", "adress": "Adress", "geoLocation": "GeoLocation", "phone": "Phone", "website": "Website", \
                "email": "Email", "category": "Category", "cropCode": "Crops", "state": "state"}
    for i in range(0, len(df)):
        data = dict()
        for key in dataItems.keys():
            value = df[dataItems[key]][i]
            if key == "geoLocation":
                dec = value.find(",")
                data[key] = [value[0:dec],value[dec+1:]]
            elif df[dataItems[key]][i] != "" and value != "N/V":
                # check if I need the u'id' here for example. If yes do that before dataItems[key].
                data[key] = value
        addFarm(data)
        
def addFarmsToJson(df):
    dataItems = {"id": "Index", "name": "Name", "adress": "Adress", "geoLocation": "GeoLocation", "phone": "Phone", "website": "Website", \
                "email": "Email", "category": "Category", "cropCode": "Crops", "state": "state"}
    
    array = list()
    for i in range(0, len(df)):
        data = dict()
        for key in dataItems.keys():
            value = df[dataItems[key]][i]
            if key == "geoLocation":
                dec = value.find(",")
                data[key] = [float(value[dec+1:]),float(value[0:dec])]
            elif df[dataItems[key]][i] != "" and value != "N/V":
                # check if I need the u'id' here for example. If yes do that before dataItems[key].
                data[key] = value
        array.append(data)
    with open('Data/farms.json', 'w', encoding='utf-8') as f:
        json.dump(array, f, ensure_ascii=False, indent=4)
        
def queryFarms(queryValue, queryField = u"Name", operator = u"=="):
    docs = db.collection(u'farms').where(queryField, operator, queryValue).stream()

    for doc in docs:
        print(u'{} => {}'.format(doc.id, doc.to_dict()))
        
def addGroup(data):
    # before adding, check if another value with the same Name & Adress exists. Right now we just add another
    dbRef = db.collection(u'groups').document()
    dbRef.set(data)
        
def addGroupsFromDF(df):
    df["id"] = df["id"].apply(lambda x: str(x))
    for i in range(0, len(df)): 
        farmsIncluded = [x.strip() for x in df.farmsIncluded[i].split(',')]
        farmsIncluded[0] = farmsIncluded[0][1:]
        farmsIncluded[-1] = farmsIncluded[-1][:-1]
        data = {
        u'id': df.id[i],
        u'location': df.location[i],
        u'farmsIncluded': farmsIncluded,
        }
        
        addGroup(data)
        
def addGroupsToJson(df):
    df["id"] = df["id"].apply(lambda x: str(x))
    array = list()
    for i in range(0, len(df)): 
        farmsIncluded = [x.strip() for x in df.farmsIncluded[i].split(',')]
        farmsIncluded[0] = farmsIncluded[0][1:]
        farmsIncluded[-1] = farmsIncluded[-1][:-1]
        dec = df.location[i].find(",")
        data = {
        u'id': df.id[i],
        u'geoLocation': [float(df.location[i][dec+1:]),float(df.location[i][0:dec])],
        u'farmsIncluded': farmsIncluded,
        }
        array.append(data
                    )
    with open('Data/Groups.json', 'w', encoding='utf-8') as f:
        json.dump(array, f, ensure_ascii=False, indent=4)

In [20]:
df = pd.read_excel("Data/TestDataGeo.xlsx")
df = df.drop(labels="Unnamed: 0", axis = 1)
df["Index"] = df["Index"].apply(lambda x: str(x))
df["Crops"] = df["Crops"].apply(lambda x: str(x))
df = df.drop_duplicates(subset=["Index"])
df = df.drop([df.columns[0],df.columns[1]], axis=1)
df = df.reset_index()

In [21]:
df[df.Index == "16472"]

,index,Index,Name,Adress,GeoLocation,Phone,Website,Category,Email,Crops,Source,DateUpdated,DateChecked,zip,state,Multiple Locations
12,13,16472,De Bortoli Wines Hunter Valley Cellar Door,"532 Wine Country Drive, Pokolbin NSW 2320","-32.7816668,151.3448978",+612 4993 8800,http://www.debortoli.com.au/,Vineyard or Winery,N/V,3,https://www.yellowpages.com.au/search/listings...,05/05/2020,2020-06-01,2320,New South Wales,NaN


In [13]:
addFarmsFromDF(df)

In [11]:
queryFarms(u"Tims Apple Place", u"Name", u"==")

In [36]:
dfGroups = pd.read_excel("Data/Groups.xlsx")

In [37]:
dfGroups.head()

,Unnamed: 0,farmsIncluded,id,location
0,0,"[16870, 16873, 16875, 16877, 16880, 16887, 168...",1,"-32.90312568717949,115.65298894358973"
1,1,"[16869, 16870, 16871, 16873, 16874, 16875, 168...",2,"-33.44937914464286,116.1302205392857"
2,2,"[16869, 16871, 16874, 16878, 16882, 16884, 168...",3,"-34.702548841176466,117.22504596470588"
3,3,"[17149, 17159, 17248, 17249, 17308, 17311]",4,"-33.682987649999994,138.58872438333336"
4,4,"[17143, 17144, 17146, 17147, 17148, 17149, 171...",5,"-35.11433341654135,139.03593302706764"


In [32]:
addGroupsFromDF(dfGroups)

In [35]:
addFarmsToJson(df)

In [38]:
addGroupsToJson(dfGroups)